# Import libraries

In [1]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain import hub
from langchain_community.document_loaders import ObsidianLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

# Set variables

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Load, split and store embeddings

In [3]:
loader = ObsidianLoader("/Users/danielmcateer/Library/Mobile Documents/iCloud~md~obsidian/Documents/Ideaverse")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant notes from Obsidian

In [4]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatGroq(temperature=0.5, groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

# Run RAG chain using LangChain

In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# Get a response from the RAG chain

In [7]:
output = {}
curr_key = None
for chunk in rag_chain_with_source.stream("Based on the ideas I've explored in my notes, I'm thinking about the following question: what would be interesting topics of study to pursue next?"):
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key != curr_key:
            print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
        else:
            print(chunk[key], end="", flush=True)
        curr_key = key
print(output)



question: Based on the ideas I've explored in my notes, I'm thinking about the following question: what would be interesting topics of study to pursue next?

context: [Document(page_content="Prompt: Use the content of my notes to formulate a coherent worldview of how I see the world. Pick out the most often mentioned perspectives that I've written about\n\nBased on your notes, your worldview seems to be centered around the following perspectives:\n\n1. **Human Mind and Consciousness**: You view the human mind as a powerful force of nature, akin to the heart's role in the body (#1). You believe in the existence of an informational layer of humanity, a collective stream of consciousness shared through the internet, which represents a new evolutionary stage (#2). You also hold that the mind is a tool for humanity, capable of creating the world we desire, but also capable of becoming our worst enemy if misused (#3, #12).", metadata={'created': 1709580839.2910812, 'last_accessed': 1709583

# Cleanup

In [8]:
vectorstore.delete_collection()